In [56]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

import json
import os
import argparse

import requests
import urllib
import urllib.request
import urllib3
from urllib3.exceptions import InsecureRequestWarning

import datetime
import time
import sys
import json
import pandas as pd
from tqdm import tqdm_notebook as tqdm

urllib3.disable_warnings(InsecureRequestWarning)

chromedriver = '../PHR/chromedriver.exe'
base_url = 'https://www.google.com/search?q='
tail_url = '&source=lnms&tbm=isch'

def download_google_staticimages(search_url,dirs):

    options = webdriver.ChromeOptions()
    options.add_argument('--no-sandbox')
    #options.add_argument('--headless')

    try:
        browser = webdriver.Chrome(chromedriver, options=options)
    except Exception as e:
        print(f'No found chromedriver in this environment.')
        print(f'Install on your machine. exception: {e}')
        sys.exit()

    browser.set_window_size(1280, 1024)
    browser.get(search_url)
    time.sleep(1)

    print(f'Getting you a lot of images. This may take a few moments...')

    element = browser.find_element_by_tag_name('body')

    # Scroll down
    #for i in range(30):
    for i in range(50):
        element.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.3)

    try:
        browser.find_element_by_id('smb').click()
        for i in range(50):
            element.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.3)
    except:
        for i in range(10):
            element.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.3)

    print(f'Reached end of page.')
    time.sleep(0.5)
#     print(f'Retry')
    time.sleep(0.5)

    # Below is in japanese "show more result" sentences. Change this word to your lanaguage if you require.
#     browser.find_element_by_xpath('//*[@id="islmp"]/div/div/div/div/div[5]/input').click()

#     element = browser.find_element_by_xpath('//*[@id="islmp"]/div/div/div/div/div[5]/input')
#     browser.execute_script("arguments[0].click();", element)
    try:
        browser.find_element_by_xpath('//*[@id="islmp"]/div/div/div/div/div[5]/input').send_keys(Keys.ENTER)
        
        # Scroll down 2
        for i in range(50):
            element.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.3)

        try:
            browser.find_element_by_id('smb').click()
            for i in range(50):
                element.send_keys(Keys.PAGE_DOWN)
                time.sleep(0.3)
        except:
            for i in range(10):
                element.send_keys(Keys.PAGE_DOWN)
                time.sleep(0.3)
  
    except Exception as e:
        print('Error at {} for {}'.format( dirs.split('/')[-1], e))


    #elements = browser.find_elements_by_xpath('//div[@id="islrg"]')
    #page_source = elements[0].get_attribute('innerHTML')
    page_source = browser.page_source 

    soup = BeautifulSoup(page_source, 'lxml')
    images = soup.find_all('img')

    urls = []
    for image in images:
        try:
            url = image['data-src']
            if not url.find('https://'):
                urls.append(url)
        except:
            try:
                url = image['src']
                if not url.find('https://'):
                    urls.append(image['src'])
            except Exception as e:
                print(f'No found image sources.')
                print(e)

    count = 0
    if urls:
        for url in urls[1:]: #0은 구글로고
            try:
                res = requests.get(url, verify=False, stream=True)
                rawdata = res.raw.read()
                with open(os.path.join(dirs, 'img_' + str(count) + '.jpg'), 'wb') as f:
                    f.write(rawdata)
                    count += 1
            except Exception as e:
                print('Failed to write rawdata.')
                print(e)

    browser.close()
    return count

# Main block
def crawler(keywords):
    
    search_url = base_url
    keywords_list = [x.strip() for x in keywords.split(',')]
    for idx,  keyword in enumerate(keywords_list):
        if idx ==0:
            search_url = search_url+keyword
        else :
            search_url = search_url+ '+' + keyword
        
    search_url = search_url + tail_url
    
    dirs = 'images/{}'.format(keywords_list[0]) 
    if not os.path.exists(dirs):
        os.mkdir(dirs)
        
    maxcount = 1000


   
    t0 = time.time()
    count = download_google_staticimages(search_url, dirs)
    t1 = time.time()

    total_time = t1 - t0
    print(f'\n')
    print(f'Download completed. [Successful count = {count}].')
    print(f'Total time is {str(total_time)} seconds.')

# if __name__ == '__main__':
#     main()

In [62]:
crawling_words = pd.read_excel('식사일기_서비스_이미지_크롤링.xlsx', sheet_name='데이터 수집필요', header=None, names=['재료', 2,3,4] )

In [16]:
for food in tqdm(crawling_words['재료'][125:]):
    crawler(food)

C:\Users\NB-0242\Anaconda3\envs\ximz\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry
Error at 새우 for Message: element not interactable
  (Session info: chrome=81.0.4044.113)



Download completed. [Successful count = 183].
Total time is 67.5463604927063 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 562].
Total time is 142.08544993400574 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 531].
Total time is 140.3233118057251 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 770].
Total time is 188.51942420005798 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 621].
Total time is 153.86043453216553 seconds.
Getting you a lot of images.

Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 487].
Total time is 134.009535074234 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 835].
Total time is 190.2690737247467 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 613].
Total time is 153.3288552761078 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 647].
Total time is 161.85307097434998 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 432].
Total time is 125.22008490562439 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successfu

Retry


Download completed. [Successful count = 600].
Total time is 148.3914921283722 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 693].
Total time is 163.63727164268494 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 609].
Total time is 183.53107476234436 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 606].
Total time is 176.76417636871338 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 742].
Total time is 179.96471071243286 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 756].
Total time is 173.78912615776062 seconds.
Getting you a lot of

Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 723].
Total time is 174.65397119522095 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 440].
Total time is 135.02862930297852 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 784].
Total time is 184.55013227462769 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 650].
Total time is 168.29466462135315 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 714].
Total time is 169.93706226348877 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Succe

...

Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 584].
Total time is 155.75537419319153 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 782].
Total time is 178.50950717926025 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 776].
Total time is 184.59523224830627 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 624].
Total time is 176.13585305213928 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 395].
Total time is 125.63492751121521 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Succe

Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 803].
Total time is 182.85487365722656 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 745].
Total time is 184.3578531742096 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 540].
Total time is 139.7771418094635 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 643].
Total time is 185.2694160938263 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 675].
Total time is 172.01587963104248 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successf

Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 732].
Total time is 185.74115681648254 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 764].
Total time is 188.06392693519592 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 650].
Total time is 158.58081698417664 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 806].
Total time is 184.16238713264465 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 688].
Total time is 175.8207278251648 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Succes

Retry


Download completed. [Successful count = 481].
Total time is 140.67769837379456 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 703].
Total time is 247.63160300254822 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 724].
Total time is 164.59671354293823 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 619].
Total time is 147.31893157958984 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 815].
Total time is 281.52494072914124 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 661].
Total time is 155.54093742370605 seconds.
Getting you a lot o

Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 735].
Total time is 321.79718255996704 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 749].
Total time is 306.5549876689911 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 762].
Total time is 180.05472993850708 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 648].
Total time is 159.38864827156067 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Successful count = 852].
Total time is 333.8110761642456 seconds.
Getting you a lot of images. This may take a few moments...
Reached end of page.
Retry


Download completed. [Success

In [4]:
crwaling_words

,재료,2,3,4
0,무,NaN,NaN,NaN
1,"감, 연시",NaN,NaN,NaN
2,감자,NaN,NaN,NaN
3,"갑오징어, 데친것",NaN,NaN,NaN
4,강낭콩,NaN,NaN,NaN
...,...,...,...,...
552,참나물무침,NaN,NaN,NaN
553,세발나물무침,NaN,NaN,NaN
554,가죽나물무침,NaN,NaN,NaN
555,뽕잎나물돼지고기무침,NaN,NaN,NaN


---------

## KOR -> ENG translation crawling

In [58]:
# with PAPAGO
client_id = '#
client_secret = '#'

def papago_translator(query):
    enc_text = urllib.parse.quote(query)
    data = 'source=ko&target=en&text=' + enc_text
    url = 'https://openapi.naver.com/v1/papago/n2mt'
    request = urllib.request.Request(url)
    request.add_header('X-Naver-Client-Id', client_id)
    request.add_header('X-Naver-Client-Secret', client_secret)
    response = urllib.request.urlopen(request, data = data.encode('utf-8'))
    rescode = response.getcode()
#     return response
    if rescode==200:
        response_body = response.read()
        res_json = json.loads(response_body.decode('utf-8'))
        
        return res_json['message']['result']['translatedText']
    else:
        print('Error code : ', rescode)
        raise 

In [66]:
crawling_words['재료_영어'] = crawling_words['재료'].map(papago_translator)

In [67]:
crawling_words

,재료,2,3,4,재료_영어
0,무,NaN,NaN,NaN,Radish
1,"감, 연시",NaN,NaN,NaN,"persimmon, soft persimmon"
2,감자,NaN,NaN,NaN,Potato
3,"갑오징어, 데친것",NaN,NaN,NaN,a cuttlefish
4,강낭콩,NaN,NaN,NaN,kidney bean
...,...,...,...,...,...
552,참나물무침,NaN,NaN,NaN,seasoned oak
553,세발나물무침,NaN,NaN,NaN,seasoned tripe greens
554,가죽나물무침,NaN,NaN,NaN,seasoned leather vegetables
555,뽕잎나물돼지고기무침,NaN,NaN,NaN,Seasoned mulberry leaf veggies pork


## 영어로 없는 메뉴가 많아서 포기 ;